In [2]:
import urllib.request, json 
import requests
import pandas as pd
import numpy as np
from scipy.stats import linregress
import psycopg2

In [3]:
try:
    connect_str = "dbname='d831qh80eusrfh' user='nsuzweabkeyhdi' host='ec2-54-247-117-145.eu-west-1.compute.amazonaws.com' " + \
                  "password='db6fda2fb0ad1826fbc32e0921a789ca0b1b9a6fc52d59a16b11571c6bb69674'"
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    # create a psycopg2 cursor that can execute queries
    cursor = conn.cursor()
    # run a SELECT statement - no data in there, but we can try it
    cursor.execute("""select experiment_id,qpcr_plate_id,(case when col_num='01' or col_num='02'  then 'a'
		   when col_num='03' or col_num='04'  then 'b'
		   when col_num='05' or col_num='06'  then 'c'
		   when col_num='07' or col_num='08'  then 'd'
		   when col_num='09' or col_num='10'  then 'e'
		   when col_num='11' or col_num='12'  then 'f'
		  end) as colgrp,ARRAY_AGG(DISTINCT qpcr_well) as wells from (select *,substring(qpcr_well,2,2) as col_num from app_qpcrresultsmodel) as temp group by experiment_id,qpcr_plate_id,colgrp""")
    well_by_id_assay = cursor.fetchall()
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [4]:
rows = ['NTC','50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','1000.0 cp ',
         '50.0 ng HgDNA ','100.0 cp ','10000.0 cp ','500.0 ng HgDNA ']
ind = ['Expt','qPCR Plate','Reps']
dep = ['ID Template Name','#Ct Pos','#Ct Valid', '#Tm Specif', '#Tm PD', '#Tm NS', 'Mean Ct', 'Min Ct',
       'Max Ct', 'Min Tm1', 'Max Tm1','Specif ng/ul Pos', 'Mean Specif ng/ul','Mean NS ng/ul', 'Mean PD ng/ul'] 

In [5]:
def list_to_string(value):
    if not isinstance(value,float):
        return "".join(str(e) for e in value)
    return None

In [6]:
def dataframe_list_to_string(df):
    list_cols = ['ID Assay Conc.','ID Assay Name','ID Human Conc.','ID Human Name',
               'ID Template Conc.','ID Template Name','PA Assay Conc.','PA Assay Name',
               'PA Human Conc.','PA Human Name','PA Template Conc.','PA Template Name']
    for column in list_cols:
        if column in df.columns:
            df[column] = df[column].apply(list_to_string)
    return df

In [7]:
def find_sample(row):    
    name = ''
    if row['ID Template Conc.']:
        name += '%s cp '%row['ID Template Conc.']
    if row['ID Human Conc.']:
        name += '%s ng HgDNA '%row['ID Human Conc.']
    if 'PA Template Conc.'in row and row['PA Template Conc.']:
        name += 'PA %s cp '%row['PA Template Conc.']
    if 'PA Human Conc.' in row and row['PA Human Conc.']:
        if 'PA' not in name:
            name += 'PA %s ng HgDNA '%row['PA Human Conc.']
        else:
            name += '%s ng HgDNA '%row['PA Human Conc.']
    if name =='':
        name = 'NTC'
    return name
    

In [8]:
def get_master_summary_data(experiments,well_by_id_assay):
    master_table = []
    summary_table = []
    for (experiment_id,qpcr_plate_id,grp,wells) in well_by_id_assay:        
#         experiment_id = record['experiment_id']
#         qpcr_plate_id = record['qpcr_plate_id']
        if experiment_id in experiments:
#             wells =json.dumps(record['wells'].split(","))    
            _url = "https://assay-screening.herokuapp.com/api/well-results/?expt={}&plate_id={}&wells={}".format(experiment_id,qpcr_plate_id,wells)
            response =json.loads(requests.get(_url).text)
            master_table = master_table + response['master_table']
            summary_table = summary_table + response['summary_table']
    return master_table,summary_table

In [9]:
def save_master_table(df):
    possible_cols = ['qPCR Plate','qPCR Well','LC Plate','LC Well','ID Assay Name','ID Template Name',
    'ID Template Conc.','ID Human Conc.','PA Assay Name','PA Template Name',
    'PA Template Conc.','PA Human Conc.','Ct','Ct Call','Tm1', 'Tm2', 'Tm3', 'Tm4','Tm Specif','Tm NS',
    'Tm PD']
    cols_in_db = [col for col in possible_cols if col in df.columns]
    df[cols_in_db].to_csv('master.csv')

In [10]:
def calc_eff(row):
    try:                
        expected_cols = ['50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','100.0 cp ','1000.0 cp ','10000.0 cp ']
        cols = list(filter(lambda x : (x,'Mean Ct') in row.index ,expected_cols))        
        idx = pd.IndexSlice
        y = np.array([float(n) for n in row.loc[idx[cols, 'Mean Ct']].values])
        x = np.log10([float(n.split()[0]) for n in cols])               
        mask = ~np.isnan(x) & ~np.isnan(y)        
        
        lin_fit = linregress(x[mask], y[mask])
        eff = (10**(-1 / lin_fit.slope) - 1) * 100
    except ValueError as ve:        
        eff = 0
        
    return eff

def calc_r2(row):
    try:        
        expected_cols = ['50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','100.0 cp ','1000.0 cp ','10000.0 cp ']
        cols = list(filter(lambda x : (x,'Mean Ct') in row.index ,expected_cols))
        idx = pd.IndexSlice
        row = row.sort_index()
        y = np.array([float(n) for n in row.loc[idx[cols, 'Mean Ct']].values])
        x = np.log10([float(n.split()[0]) for n in cols])
        
        mask = ~np.isnan(x) & ~np.isnan(y)
        
        slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
        r2 = r_value**2
    except ValueError as ve:
        r2 = np.nan
        
    return r2

def removeErrFrames(frames):
    expt_list=[]
    new_df=frames[0].T
    new_df=new_df.drop(new_df.index[0])
    for i in range(1,len(frames)):         
        try:
            new_df=new_df.append(frames[i].T)
        except:
            expt_list.append(i)
    for index in sorted(expt_list, reverse=True):
        print(frames[index])
        del frames[index]
    return frames
    

In [11]:
def get_summary_by_sample(df,rows,ind,dep):
    idx = pd.IndexSlice
#     df = df[df['qPCR Plate']!='A81_E214_1_ID'] 
    frames = []    
# ['Expt','plate_id','Group','templates']
    for n, grp in df.groupby(['qPCR Plate','ID Assay Name']):    
        a = pd.DataFrame(grp.set_index('sample').loc[rows].stack())
        a = a.sort_index(level=[0, 1])
        a = a.sort_index()
        b = a.loc[idx['NTC', ind], :]
        b.index = pd.MultiIndex.from_product([['Common'], b.index.get_level_values(1)])
        c = a.loc[idx[:, dep], :]

        df = pd.concat([b, c])
        df.columns = pd.MultiIndex.from_tuples([n])
        #df = df.rename(columns={0: n[2]})
        frames.append(df)
    return frames

In [12]:
def calculate_eff_r2(df,rows):
    frames=removeErrFrames(df)
    frames_df = pd.concat(frames, axis=1).reindex(rows, level=0)
    frames_df = frames_df.sort_index().T
    frames_df[('Common', 'Efficiency')] = frames_df.apply(calc_eff, axis=1)
    frames_df[('Common', 'R Squared')] = frames_df.apply(calc_r2,axis=1)
    return frames_df

In [12]:
def get_summary(experiments,rows,ind,dep,well_by_id_assay):
#     with urllib.request.urlopen("https://assay-screening.herokuapp.com/api/well-aggregate/") as url:
#         well_by_id_assay = json.loads(url.read().decode())
    master_table,summary_table = get_master_summary_data(experiments,well_by_id_assay)
    master_df = dataframe_list_to_string(pd.DataFrame(master_table))
    summary_df = dataframe_list_to_string(pd.DataFrame(summary_table))
    summary_df['sample'] = summary_df.apply(find_sample,axis=1)
    save_master_table(master_df)
    summary_by_sample=get_summary_by_sample(summary_df,rows,ind,dep)
    summary_with_eff_r2 = calculate_eff_r2(summary_by_sample,rows)
    summary_with_eff_r2 = summary_with_eff_r2.sort_index(axis=1, level=[0, 1])
    summary_with_eff_r2.to_csv('group_by_columns.csv')
    return summary_with_eff_r2

In [1]:
experiments = ['A81_E303', 'A81_E307', 'A81_E311', 'A81_E312', 'A81_E316', 'A81_E317', 'A81_E318',
               'A81_E321', 'A81_E323', 'A81_E325', 'A81_E326', 'A81_E329']

get_summary(experiments,rows,ind,dep,well_by_id_assay)